# Using config files

Configuration is a huge problem for machine learning code, because you may want to expose almost any detail of any function as a hyperparameter. The setting you want to expose might be arbitrarily 
far down in your call stack. Default values also become hard to change without breaking backwards compatibility.

To solve this problem, Thinc provides a config system that lets you easily describe arbitrary trees of objects. The objects can be created via function calls you register using a simple decorator syntax. The config can include values like hyperparameters or training settings (whatever you need), or references to functions and the values of their arguments.

Thinc will parse the configuration file as a dict and fill in the references to values defined in other sections.
For example, `${hyper_params:learn_rate}` is substituted with `0.001` in the next cell.

Keys starting with `@` are references to **registered functions**.



In [5]:
from thinc.api import Config, registry

EXAMPLE_CONFIG1 = """
[hyper_params]
learn_rate = 0.001

[optimizer]
@optimizers = "Adam.v1"
learn_rate = ${hyper_params:learn_rate}
"""

config1 = Config().from_str(EXAMPLE_CONFIG1)
config1

{'hyper_params': {'learn_rate': 0.001},
 'optimizer': {'@optimizers': 'Adam.v1', 'learn_rate': 0.001}}

Configs can also define **nested blocks** using the `.` notation. In this example, `optimizer.learn_rate` defines the `learn_rate` argument of the `optimizer` block. Instead of a float, the learning rate can also be a generator – for instance, a linear warm-up rate:

In [7]:
EXAMPLE_CONFIG2 = """
[optimizer]
@optimizers = "Adam.v1"

[optimizer.learn_rate]
@schedules = "warmup_linear.v1"
initial_rate = 2e-5
warmup_steps = 1000
total_steps = 10000
"""

config2 = Config().from_str(EXAMPLE_CONFIG2)
config2

{'optimizer': {'@optimizers': 'Adam.v1',
  'learn_rate': {'@schedules': 'warmup_linear.v1',
   'initial_rate': 2e-05,
   'warmup_steps': 1000,
   'total_steps': 10000}}}

Thinc will create the optimizer and pass in the schedule as the `learn_rate` argument.

In [1]:
import thinc
from thinc.api import Config, registry
from typing import Union, Iterable


class MyCoolOptimizer():
    def __init__(self, learn_rate: float, gamma: Iterable[float]):
        self.eta = learn_rate
        self.gamma = gamma
    
    def __repr__(self):
        return f"MyCoolOptimizer(eta={self.eta}, gamma={self.gamma})"

        
op = MyCoolOptimizer(2.0, 3.0)
print(op)

MyCoolOptimizer(eta=2.0, gamma=3.0)


In [2]:
@thinc.registry.optimizers.register("my_cool_optimizer.v1")
def make_my_optimizer(learn_rate: Union[float, Iterable[float]], gamma: float):
    return MyCoolOptimizer(learn_rate, gamma)

@thinc.registry.schedules("my_cool_decaying_schedule.v1")
def decaying(base_rate: float, decay: float, *, t: int = 0) -> Iterable[float]:
    while True:
        yield base_rate * (1.0 / (1.0 + decay * t))
        t += 1

In [3]:
# Later you can retrieve your function by name:
create_optimizer = thinc.registry.optimizers.get("my_cool_optimizer.v1")
create_optimizer

<function __main__.make_my_optimizer(learn_rate:Union[float, Iterable[float]], gamma:float)>

In [4]:
config = Config().from_disk("example.ini")
C = registry.make_from_config(config)
C['optimizer']

RegistryError: Cant't find 'my_cool_schedule.v1' in registry thinc -> schedules. Available names: compounding.v1, constant.v1, constant_then.v1, cyclic_triangular.v1, decaying.v1, my_cool_decaying_schedule.v1, slanted_triangular.v1, warmup_linear.v1

# Variable positional arguments example

In some situations, your registered function may accept variable positional arguments. In your config, you can then use * to define a list of values:

```
[schedule]
@schedules = "my_cool_schedule.v1"
* = [0.05, 0.1, 0.25, 0.75, 0.9]
final = 1.0
```


> Type hints for variable arguments should always describe the type of the individual arguments.


In [ ]:
@thinc.registry.schedules("my_cool_schedule.v1")
def schedule(*steps: float, final: float = 1.0) -> Iterable[float]:
    yield from steps
    while True:
        yield final